In [42]:
import pandas as pd

# user_ratings = pd.read_csv('user_ratings.csv')
genres = pd.read_csv('genres.csv')
df = pd.read_csv('user_ratings_genres.csv')
users = pd.read_csv('users.csv')

In [44]:
df = df.iloc[:, 1:]
users = users.iloc[:, 1:]

In [46]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [47]:
df.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'UserID', 'Age',
       'Occupation', 'Zip-code', 'Gender_M', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Mystery', 'Romance', 'Science Fiction',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [59]:
new_X.columns

Index(['user_id', 'movie_id', 'timestamp', 'UserID', 'Age', 'Occupation',
       'Zip-code', 'Gender_M', 'Adventure_x', 'Animation_x', 'Comedy_x',
       'Crime_x', 'Documentary_x', 'Drama_x', 'Family_x', 'Fantasy_x',
       'History_x', 'Horror_x', 'Mystery_x', 'Romance_x', 'Science Fiction_x',
       'Thriller_x', 'War_x', 'Western_x', 'Adventure_y', 'Animation_y',
       'Comedy_y', 'Crime_y', 'Documentary_y', 'Drama_y', 'Family_y',
       'Fantasy_y', 'History_y', 'Horror_y', 'Mystery_y', 'Romance_y',
       'Science Fiction_y', 'Thriller_y', 'War_y', 'Western_y', 'Adventure',
       'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'History', 'Horror', 'Mystery', 'Romance', 'Science Fiction',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [49]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import train_test_split
from sklearn.model_selection import train_test_split

X = df.drop(["rating"], axis=1)
y = df["rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=142, stratify = y)

forest_amount = RandomForestClassifier(n_estimators=30, max_depth=10)
forest_amount.fit(X_train, y_train)
print("Train Accuracy", forest_amount.score(X_train, y_train)) # Accuracy of training data
print("Test Accuracy", forest_amount.score(X_test, y_test)) # Accuracy of test data

# plot_feature_importances(forest_amount) # plot using function above
def plot_feature_importances(model):
   '''plots the importance of each feature.  Useful for something like KNN so you can tell which features are useful '''
   n_features = X_train.shape[1]
   plt.figure(figsize=(8,8))
   plt.barh(range(n_features), model.feature_importances_, align='center')
   plt.yticks(np.arange(n_features), X_train.columns.values)
   plt.xlabel("Feature importance")
   plt.ylabel("Feature")

Train Accuracy 0.37561411344350154
Test Accuracy 0.36090913670635255


In [50]:
# fit all data to model 
forest_amount = RandomForestClassifier(n_estimators=30, max_depth= 9)
forest_amount.fit(X, y)
print("Accuracy", forest_amount.score(X, y)) 

Accuracy 0.3662202075840782


In [51]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [60]:
# make new users dataset
new_X = pd.read_csv('request_df.csv')

new_X = new_X.iloc[:, 1:]
new_X = new_X.merge(users, left_on="user_id", right_on='UserID', how='left')
new_X['Zip-code']=new_X["Zip-code"].apply(lambda x: x[:5])
new_X["Zip-code"]=new_X["Zip-code"].astype(int)
new_X = pd.get_dummies(new_X, columns=['Gender'], drop_first=True)
new_X = new_X.drop(['rating'], axis=1)

In [62]:
new_X = new_X.merge(genres, on='movie_id', how='left')
new_X = new_X.fillna(0.0)
new_X.head()

,user_id,movie_id,timestamp,UserID,Age,Occupation,Zip-code,Gender_M,Adventure,Animation,...,Family,Fantasy,History,Horror,Mystery,Romance,Science Fiction,Thriller,War,Western
0,6040,2019,956678777.0,6040,25,6,11106,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6040,759,956679248.0,6040,25,6,11106,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6040,2858,956679275.0,6040,25,6,11106,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6040,246,956679413.0,6040,25,6,11106,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6040,1617,956679473.0,6040,25,6,11106,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# make predictions on new users

predictions = forest_amount.predict(new_X)

In [64]:
# make a df of predictions

predictions_df = pd.DataFrame({'userId': new_X['UserID'], 'movieId': new_X['movie_id'], 'predictions': predictions})



In [65]:
predictions_df.predictions.value_counts()

4    271440
5      4629
3      4165
2        26
Name: predictions, dtype: int64

In [66]:
df['rating'].value_counts()

4    124579
3     94781
5     82450
2     38621
1     20483
Name: rating, dtype: int64

In [67]:
predictions_df.to_csv('predictions.csv')